# Checking lab data

In [ ]:
import os
import sys
from functools import reduce
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets

sys.path.insert(0, '../../')
from brickstudy import foundation

In [ ]:
lab_data = pd.read_csv('BRICK_lab_data_update_18_03_2024.csv', sep= ';', encoding='latin1')


In [ ]:

lab_data

In [ ]:
lab_data['studieID'] = lab_data['studieID'].astype(str)
lab_data['studieID'] = lab_data['studieID'].str.replace("-", "_")

In [ ]:
lab_data.dtypes

In [ ]:
lab_data.head(3)

In [ ]:
lab_data.code_code_original.unique()

In [ ]:
# make tables for each code_code_original

code_codeCRP = lab_data[lab_data['code_code_original'] == 'CRP']

In [ ]:
code_codeCRP

In [ ]:
lab_data2 = pd.read_excel('../../secret_data/Brick_lab_20230914.xlsx')

In [ ]:
lab_data2

In [ ]:
lab_data2.groupby('studieID').code_code_original

In [ ]:
lab_data2.groupby?